In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from ydata_profiling import ProfileReport
from autogluon.tabular import TabularPredictor,TabularDataset
import os

d:\worker_code\.venvgis\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
d:\worker_code\.venvgis\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [2]:
model_save_path = 'D:\ArcgisData\pred_soildeep\pred_model'

In [3]:
# 读取数据
file_path = r"F:\cache_data\pre_property_table\dy\feature_deep_dy.csv"
data = pd.read_csv(file_path)
data


,HD,纬度,经度,岩石类型,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,...,VAP,WIND,BIO,LON,LAT,PH,DZ,DL,MRRTF,MRVBF
0,厚,26.027272,107.435591,灰岩,862.30005,0.587879,4.686081,861.02300,1.277039,0.0,...,1.544167,1.675000,192.94983,443505.0,2879980.0,5.794220,3,3,0.000015,0.000309
1,薄,26.021210,107.438581,灰岩,845.00000,0.865729,0.821099,841.62115,3.378845,0.0,...,1.560000,1.650000,192.69456,443800.0,2879305.0,5.858460,3,37,0.168167,0.509042
2,厚,26.018370,107.442429,泥岩,870.40000,0.589348,4.615138,860.02080,10.379211,0.0,...,1.550000,1.641667,192.49423,444185.0,2878990.0,5.354382,3,7,0.003655,0.005572
3,薄,26.043651,107.445306,灰岩,895.00000,0.565625,4.637900,887.03290,7.967102,0.0,...,1.534167,1.683333,192.67345,444485.0,2881790.0,5.464322,3,16,0.011689,0.044152
4,厚,26.038790,107.440876,灰岩,917.90000,0.999877,1.157107,896.93024,20.969788,0.0,...,1.534167,1.683333,192.75296,444040.0,2881250.0,5.551921,3,16,0.027246,0.786321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4082,厚,26.132766,107.700090,第四系红黏土,943.40000,0.463589,5.878293,893.93030,49.469727,0.0,...,1.529167,1.683333,193.17662,470005.0,2891580.0,5.155038,5,12,0.194916,0.505479
4083,厚,26.143265,107.685564,第四系红黏土,811.00000,0.679758,5.431235,761.65750,49.342530,0.0,...,1.570000,1.608333,192.02028,468560.0,2892745.0,5.083570,5,1,2.963596,0.588836
4084,厚,26.147965,107.685542,第四系红黏土,793.70000,0.701160,4.434078,760.73140,32.968628,0.0,...,1.570000,1.608333,192.02028,468555.0,2893265.0,5.326052,5,7,0.496160,0.457307
4085,厚,26.145499,107.682173,第四系红黏土,824.80000,0.549312,5.853435,762.59937,62.200623,0.0,...,1.570000,1.608333,192.55142,468220.0,2892995.0,5.149998,5,3,0.334580,0.551369


In [4]:
data.drop_duplicates(inplace=True)
data.describe()

,纬度,经度,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,...,VAP,WIND,BIO,LON,LAT,PH,DZ,DL,MRRTF,MRVBF
count,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,...,4087.000000,4087.000000,4087.000000,4087.000000,4.087000e+03,4087.000000,4087.000000,4087.000000,4.087000e+03,4.087000e+03
mean,26.142046,107.479359,983.718594,0.878765,3.122764,963.519669,20.769652,0.570728,0.594842,15.798080,...,1.500277,1.717319,192.456817,447933.947884,2.892686e+06,5.614816,5.310741,11.082701,3.099244e-01,5.280551e-01
std,0.121163,0.159751,183.031749,0.313091,1.703414,177.097324,24.577441,4.156249,6.491944,16.698154,...,0.061181,0.091248,1.550586,15985.402055,1.341216e+04,0.407939,3.478180,11.397719,6.098156e-01,7.776929e-01
min,25.857301,107.129531,572.100000,0.033098,0.000000,573.533700,-0.150024,0.000000,-55.795593,0.000000,...,1.280000,1.541667,187.725100,412900.000000,2.861150e+06,4.447507,2.000000,0.000000,1.132427e-14,7.038814e-14
25%,26.054691,107.366670,852.700000,0.678908,1.742258,840.099925,4.390625,0.000000,-0.845028,7.183879,...,1.469167,1.658333,191.612875,436657.500000,2.882990e+06,5.337979,3.000000,5.000000,1.111643e-02,2.930174e-02
50%,26.128644,107.467712,943.900000,0.869113,3.124044,924.530940,13.090210,0.000000,0.655742,13.012947,...,1.512500,1.691667,192.259630,446755.000000,2.891205e+06,5.545370,4.000000,7.000000,1.068349e-01,2.671965e-01
75%,26.233529,107.613450,1082.150050,1.094983,4.490529,1057.685650,28.027099,0.000000,2.322910,20.619241,...,1.544167,1.766667,193.280820,461367.500000,2.902868e+06,5.825024,6.000000,9.000000,2.982050e-01,5.776829e-01
max,26.418161,107.777844,1660.200000,2.131159,6.283186,1628.765500,289.642640,60.401306,77.550780,455.575260,...,1.623333,2.066667,197.809980,477815.000000,2.923275e+06,7.361616,14.000000,42.000000,4.961693e+00,5.093605e+00


In [6]:
data.columns

Index(['HD', '纬度', '经度', '岩石类型', 'DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature',
       'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity',
       'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE',
       'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT', 'PH', 'DZ', 'DL',
       'MRRTF', 'MRVBF'],
      dtype='object')

In [16]:
data = data[['HD','DEM','Aspect','PlanCurvature','ProfileCurvature','Slope','TopographicWetnessIndex','ValleyDepth','ndvi','PCA_0','MRVBF']]
data['HD'] = data['HD'].astype('category')
# data['DL'] = data['DL'].astype('category')
data.dtypes

HD                         category
DEM                         float64
Aspect                      float64
PlanCurvature               float64
ProfileCurvature            float64
Slope                       float64
TopographicWetnessIndex     float64
ValleyDepth                 float64
ndvi                        float64
PCA_0                       float64
MRVBF                       float64
dtype: object

In [17]:
train_data, test_data = train_test_split(data,test_size=0.3,random_state=42)

In [18]:
label = 'HD'

In [19]:
predictor = TabularPredictor(label=label,eval_metric='f1_weighted',path=os.path.join(model_save_path,"test0612"),problem_type='multiclass')
predictor.fit(train_data,tuning_data=test_data,time_limit=10800)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "D:\ArcgisData\pred_soildeep\pred_model\test0612"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       36.77 GB / 63.81 GB (57.6%)
Disk Space Avail: 

[1000]	valid_set's multi_logloss: 2.05592	valid_set's f1_weighted: 0.584248


	0.5855	 = Validation score   (f1_weighted)
	5.75s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 1079.99s of the 10780.39s of remaining time.
	Ensemble Weights: {'LightGBMXT': 0.524, 'LightGBM': 0.238, 'XGBoost': 0.095, 'KNeighborsDist': 0.048, 'RandomForestEntr': 0.048, 'LightGBMLarge': 0.048}
	0.5931	 = Validation score   (f1_weighted)
	0.22s	 = Training   runtime
	0.02s	 = Validation runtime
AutoGluon training complete, total runtime = 19.87s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("D:\ArcgisData\pred_soildeep\pred_model\test0612")


In [20]:
predictor.leaderboard(train_data,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,1.000000,0.572051,f1_weighted,0.067677,0.035050,0.548240,0.067677,0.035050,0.548240,1,True,7
1,ExtraTreesEntr,1.000000,0.554820,f1_weighted,0.082869,0.034996,0.382747,0.082869,0.034996,0.382747,1,True,10
2,RandomForestGini,1.000000,0.569307,f1_weighted,0.085718,0.034020,0.499990,0.085718,0.034020,0.499990,1,True,6
3,ExtraTreesGini,1.000000,0.557156,f1_weighted,0.100683,0.034324,0.379201,0.100683,0.034324,0.379201,1,True,9
4,LightGBMLarge,1.000000,0.585466,f1_weighted,0.145079,0.033834,5.746602,0.145079,0.033834,5.746602,1,True,13
5,KNeighborsDist,0.999301,0.547884,f1_weighted,0.009294,0.000000,0.000000,0.009294,0.000000,0.000000,1,True,2
6,WeightedEnsemble_L2,0.994395,0.593114,f1_weighted,0.350633,0.105533,11.659592,0.016600,0.016372,0.215991,2,True,14
7,LightGBMXT,0.987359,0.591326,f1_weighted,0.039796,0.004653,2.398700,0.039796,0.004653,2.398700,1,True,4
8,XGBoost,0.961363,0.578774,f1_weighted,0.047425,0.015625,1.600339,0.047425,0.015625,1.600339,1,True,11
9,LightGBM,0.893916,0.589794,f1_weighted,0.024762,0.000000,1.149719,0.024762,0.000000,1.149719,1,True,5


In [14]:
predictor.leaderboard(test_data,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.686921,0.690897,f1_weighted,0.307956,0.120581,18.975028,0.015627,0.001172,0.235604,2,True,14
1,KNeighborsDist,0.677592,0.681359,f1_weighted,0.002000,0.000000,0.004945,0.002000,0.000000,0.004945,1,True,2
2,KNeighborsUnif,0.671786,0.675511,f1_weighted,0.002999,0.004115,3.183639,0.002999,0.004115,3.183639,1,True,1
3,LightGBMLarge,0.643984,0.647712,f1_weighted,0.016642,0.007005,3.140921,0.016642,0.007005,3.140921,1,True,13
4,LightGBM,0.639975,0.643705,f1_weighted,0.053690,0.016971,2.749868,0.053690,0.016971,2.749868,1,True,5
5,XGBoost,0.639064,0.642782,f1_weighted,0.033335,0.000000,1.566693,0.033335,0.000000,1.566693,1,True,11
6,CatBoost,0.631638,0.635277,f1_weighted,0.002068,0.000000,22.899670,0.002068,0.000000,22.899670,1,True,8
7,LightGBMXT,0.624904,0.628553,f1_weighted,0.034285,0.013763,2.469623,0.034285,0.013763,2.469623,1,True,4
8,RandomForestEntr,0.620330,0.623923,f1_weighted,0.083746,0.034868,0.597418,0.083746,0.034868,0.597418,1,True,7
9,RandomForestGini,0.618264,0.621857,f1_weighted,0.109587,0.033410,0.566340,0.109587,0.033410,0.566340,1,True,6
